In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from pystac_client import Client
import stackstac
import rioxarray
from omnicloudmask import predict_from_array

/home/mlla/projects/cloud_removal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def download_tile_and_sc2l(date, tile, catalog_url, collection):
    # Connect to STAC API
    catalog = Client.open(catalog_url)

    # Search for the Sentinel-2 tile on the specified date
    search = catalog.search(
        collections=[collection],
        datetime=date,  
        query={
            "mgrs:utm_zone": {"eq": tile[:2]},
            "mgrs:latitude_band": {"eq":  tile[2:3]},
            "mgrs:grid_square": {"eq":  tile[3:5]}
        },
        #bbox=bbox,
        limit=10
    )
    # Get the first item from the search results
    items = list(search.items())
    if not items:
        raise ValueError("No items found for the specified tile and date.")
    
    item = items[0]
    # Get asset URLs
    assets = item.assets
    red_url = assets["red"].href
    green_url = assets["green"].href
    blue_url = assets["blue"].href
    nir_url = assets["nir"].href
    scl_url = assets["scl"].href

    # Open the bands using rasterio
    with rasterio.open(red_url) as red_src, \
         rasterio.open(green_url) as green_src, \
         rasterio.open(blue_url) as blue_src, \
         rasterio.open(nir_url) as nir_src:
        # Prepare output GeoTIFFs
        profile = red_src.profile
        profile.update(dtype="float32", count=3, compress="lzw")
        
        with rasterio.open(f"{tile}_rgbn.tif", "w", **profile) as dst:
            red_chunk = red_src.read(1)
            green_chunk = green_src.read(1)
            blue_chunk = blue_src.read(1)
            nir_chunk = nir_src.read(1)
            rgb_nir = np.stack([
                red_chunk,
                green_chunk,
                blue_chunk,
                nir_chunk,
            ])
            dst.write(rgb_nir.astype("float32"))
    #scl = xda_data.sel(band="scl")
    #scl.median(dim="time", skipna=True).rio.to_raster(f"{tile}_scl.tif")
    #rgb.median(dim="time", skipna=True).rio.to_raster(f"{tile}_rgb.tif")

In [7]:
# Define parameters
catalog_url = "https://earth-search.aws.element84.com/v1"
collection = "sentinel-2-l2a"
tile = "26XPF"
date = "2024-08-26"

In [11]:
def download_tile_and_scl(date, tile, catalog_url, collection):
    # Connect to STAC API
    catalog = Client.open(catalog_url)

    # Search for the Sentinel-2 tile on the specified date
    search = catalog.search(
        collections=[collection],
        datetime=date,  
        query={
            "mgrs:utm_zone": {"eq": tile[:2]},
            "mgrs:latitude_band": {"eq":  tile[2:3]},
            "mgrs:grid_square": {"eq":  tile[3:5]}
        },
        #bbox=bbox,
        limit=10
    )
    # Get the first item from the search results
    items = list(search.items())
    if not items:
        raise ValueError("No items found for the specified tile and date.")
    
    xda_data = stackstac.stack(
        items,
        rescale=False,
        epsg=4326
    )

    # Select RGB and SCL bands
    rgb = xda_data.sel(band=["red", "green", "blue", "nir"])
    scl = xda_data.sel(band="scl")

    # Median composite and reproject to EPSG:4326 if not already
    rgb_median = rgb.median(dim="time", skipna=True)

    # Filter SCL to only keep values 3, 8, 9, 10
    scl_filtered = scl.where(scl.isin([3, 8, 9, 10]))
    scl_max = scl_filtered.max(dim="time", skipna=True)

    # Ensure the data is in EPSG:4326
    rgb_median = rgb_median.rio.write_crs(4326, inplace=True)
    scl_max = scl_max.rio.write_crs(4326, inplace=True)

    # Save to GeoTIFF in EPSG:4326
    scl_max.rio.to_raster(f"{tile}_scl.tif")
    rgb_median.rio.to_raster(f"{tile}_rgb.tif")


In [10]:
download_tile_and_scl(date, tile, catalog_url, collection)